In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/taylorswiftlyrics/taylor_swift_lyrics.csv
/kaggle/input/gpt2/keras/gpt2_base_en/2/config.json
/kaggle/input/gpt2/keras/gpt2_base_en/2/tokenizer.json
/kaggle/input/gpt2/keras/gpt2_base_en/2/metadata.json
/kaggle/input/gpt2/keras/gpt2_base_en/2/model.weights.h5
/kaggle/input/gpt2/keras/gpt2_base_en/2/assets/tokenizer/merges.txt
/kaggle/input/gpt2/keras/gpt2_base_en/2/assets/tokenizer/vocabulary.json


In [11]:
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import keras_hub
import tensorflow as tf
import tensorflow.data as tf_data

In [3]:
Preprocessor = keras_hub.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length = 512,
)

In [4]:
GPT2Model = keras_hub.models.GPT2CausalLM.from_preset(
    "gpt2_base_en",
    preprocessor=Preprocessor
)

In [5]:
test = GPT2Model.generate(
    "You're one the phone with your girlfriend, she's upset.",
    max_length = 256,
)

In [7]:
print(test)

You're one the phone with your girlfriend, she's upset. You're not. She's a little upset. You don't know what she's feeling, but maybe it's the same thing that you did with your girlfriend. You can't help but be sad. You're not.

It's not that you're not upset. It's that you're a little upset, and you feel a little sad about it.

You're not.

The first thing you do is to talk.

The next thing you do is try to make sense of it.

The next time you're in your phone you're going to be a little more upset.

You're not.

You're not.

You're not.

You're not.

You're not.

The first thing you do is to try to make sense of it.

The next time you're in your phone you're going to be a little more upset.

The next time you're in your phone you're going to be a little more upset and angry at yourself, at yourself.

You're not.

You're not.

You're not.

It's not


As we can see, this GPT2 Model sucks absolute balls.

In [8]:
import pandas as pd

In [14]:
import re
remove_tags = lambda text: re.sub(r'\[.*?\]', '', text)

In [30]:
lyrics = pd.read_csv('/kaggle/input/taylorswiftlyrics/taylor_swift_lyrics.csv').dropna()

In [34]:
lyrics["Lyrics"] = lyrics["Lyrics"].apply(remove_tags)

In [37]:
dataset = tf_data.Dataset.from_tensor_slices(lyrics["Lyrics"].values)

In [40]:
dataset = dataset.batch(16).cache().prefetch(tf_data.AUTOTUNE)

In [41]:
from keras.optimizers import Adam
from keras.optimizers.schedules import PolynomialDecay
from keras.losses import SparseCategoricalCrossentropy

In [42]:
scheduler = PolynomialDecay(
    5e-5,
    decay_steps=dataset.cardinality() * 10,
    end_learning_rate=0.0,
)

In [44]:
loss = SparseCategoricalCrossentropy(from_logits=True)

In [45]:
GPT2Model.compile(
    optimizer=Adam(scheduler),
    loss=loss,
    weighted_metrics=["accuracy"]
)

In [46]:
GPT2Model.summary()

Preprocessor: "gpt2_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt2_tokenizer (GPT2Tokenizer)                                │                       Vocab size: 50,257 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gpt2_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gpt2_backbone (GPT2Backbone)  │ (None, None, 768)         │     124,439,808 │ padding_mask[0][0],        │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50257)       │      38,597,376 │ gpt2_backbone[0][0]        │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 124,439,808 (474.70 MB)

 Trainable params: 124,439,808 (474.70 MB)

 Non-trainable params: 0 (0.00 B)

In [47]:
GPT2Model.fit(dataset, epochs=10)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 157s 8s/step - accuracy: 0.4945 - loss: 2.5439
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5218 - loss: 2.3171
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5322 - loss: 2.2381
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5408 - loss: 2.1853
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5441 - loss: 2.1415
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5508 - loss: 2.1080
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5524 - loss: 2.0749
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5547 - loss: 2.0625
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5554 - loss: 2.0470
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5566 - loss: 2.0435


In [50]:
GPT2Model.fit(dataset, epochs=10)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5570 - loss: 2.0363
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5580 - loss: 2.0387
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5572 - loss: 2.0399
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5577 - loss: 2.0396
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5578 - loss: 2.0371
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5579 - loss: 2.0373
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5588 - loss: 2.0389
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5584 - loss: 2.0375
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5589 - loss: 2.0361
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5572 - loss: 2.0363


In [51]:
test = GPT2Model.generate(
    "You're one the phone with your girlfriend, she's upset.",
    max_length = 100,
)

print(test)

You're one the phone with your girlfriend, she's upset.
You're one of the few people that you know, you know what you should do
You're one of those people you can trust
You're one of those who will say, 'Hey, hey, hey'
You're one of those who are always right, you know why we're so upset, but you know why we're upset
You're a good boy, you're a good girl, you're


**I probably need a larger dataset!**